# <font color=#c51b8a>deepBreaks Applications</font>
### Modeling the phenotypes and spectral tuning sites of opsin proteins based on amino-acid sequence...  

# <font color=c994c7>Step 0: VPOD Setup w/SQLite</font>
### *The following function sets up the schema for our vizphiz database it doesn't yet exist. Otherwise it loads an existing version of VPOD stored locally.*
```

In [ ]:
#All neccessary packages to import for data process steps.
import re
import os
import datetime 
import subprocess
import shutil 
import csv
import fileinput
from pathlib import Path

In [ ]:
# edit vpod_root_path to point to the project directory
vpod_root_path = Path('../..').resolve()
vpod_scripts_path = vpod_root_path/'scripts_n_notebooks'
vpod_data_path = vpod_root_path/'vpod_data/VPOD_1.2'
from vpod_scripts.vpod_db import init_db

mydb = init_db(vpod_data_path/'vizphiz.db',vpod_data_path/'raw_database_files')

# <font color=c994c7>Step 1a: Extract Heterolgous Data Subsets From VPOD</font>
 - Output = 9 Different Data Subsets 

In [ ]:
import pandas as pd
from deepBreaks.preprocessing import read_data
from vpod_scripts.extract_vpod_datasets import extract_vpod_datasets, get_mnm_datasets

seq_report_dir, data_split_list, meta_data_list, mnm_meta_list, mnm_meta_shorthand = extract_vpod_datasets(mydb)
print(f"VPOD datasets extracted to: {seq_report_dir}")

# <font color=c994c7>Step 1b: Create 'Inferred' Physiological Data Subsets From Mine-n-Match (MNM)</font>

### These datasets Consider Physiology Data (i.e. Single-Cell Microspectrophotemtry Readings) in Combination With the Heterologus Data From VPOD
 - Output is 5 extra datasets marked 'mnm' in the same folder directory created above ^ ('i.e. - The 'wt_mnm_meta.csv' file contains VPOD's heterolgous and MNM data)

 - The MNM pipeline (which you can find in this same folder) connects sequence to it's closest MSP value based on OPTICS predictions (our command-line tool for predicting opsin lambda max)
 
 - For more information, read our publication - HERE

In [ ]:
path = './mine_n_match/mnm_data/mnm_on_mnm_on_all_dbs_2025-02-24_16-29-54'
mnm_file = f'{path}/mnm_on_vpod_acc_dbs_results_fully_filtered.csv'
mnm_data = read_data(mnm_file, seq_type = None, is_main=False)
# This function generates the 'mnm' datasets,
# which are a combination of the VPOD 'het' datasets and the corresponding 'mnm' data
data_split_list, meta_data_list = get_mnm_datasets(seq_report_dir, mnm_data, mnm_meta_list, mnm_meta_shorthand, meta_data_list, data_split_list)

# <font color=#c994c7>Step 2: Align Raw Data w/MAFFT and Format for 'deepBreaks'</font>
### REMINDER - You will need to change the directory for the 'mafft_exe' variable to the one of your own operating system!


In [ ]:
from vpod_scripts.extract_vpod_datasets import perform_mafft_alignment
mafft_exe = 'C:/Users/safra/mafft-win/mafft.bat' 
deep_breaks_input_data = perform_mafft_alignment(data_split_list, mafft_exe)

# <font color=#c994c7>Step 3: deepBreaks</font>
## THIS IS A LONG SECTION! 
 - **Output** = folder containing all results from model training, including comparison of model performances, an amino-acid site importance report + figures, and the top 5 trained models in a .pkl file format.

In [ ]:
# importing deepBreaks libraries 
from vpod_scripts.deepBreaks.utils_alt2 import get_models, get_best_aa_prop_combos, get_scores, get_params, get_empty_params, make_pipeline
from vpod_scripts.deepBreaks.preprocessing import MisCare, ConstantCare, URareCare, CustomOneHotEncoder, AminoAcidPropertyEncoder
from vpod_scripts.deepBreaks.preprocessing import FeatureSelection, CollinearCare
from vpod_scripts.deepBreaks.preprocessing import read_data
from vpod_scripts.deepBreaks.models import model_compare_cv, finalize_top, importance_from_pipe, aaprop_importance_from_pipe,  mean_importance, summarize_results
from vpod_scripts.deepBreaks.visualization import plot_scatter, dp_plot, dp_aa_prop_plot, plot_imp_model, plot_imp_all
from vpod_scripts.deepBreaks.preprocessing import write_fasta
import numpy as np
import pandas as pd
import warnings
import datetime
import os
import shutil 

In [ ]:
warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")

In [ ]:
# defining user params, file pathes, analysis type

# assign your path to folder containing all the datasplits
#path = './vpod_1.2_data_splits_2025-02-28_15-51-04'
path = f'./{seq_report_dir}'

# path to sequences of interest
seqFileName = f'{path}/wds_mnm_aligned_VPOD_1.2_het.fasta' 

# path to corresponding metadata of interest
metaDataFileName = f'{path}/wds_mnm_meta.csv' 
# only put dataset name if you want the grid-search p[timizerd models.
# otherwise, leave as 'ignore'
dataset = 'wds_mnm'
use_gs_params = False

# name of the phenotype
mt = 'Lambda_Max'

# If true, train models using electron-volts (ev) instead of normal wavelength (nm) values.
use_ev = False

# type of the sequences
seq_type = 'aa'

# type of the analysis if it is a classification model, then we put cl instead of reg
ana_type = 'reg' 

# Proportion of gaps at an amino acid site alloable in an alignment before dropping column completely
gap_threshold = 0.5

# Whether or not you want to drop the reference sequence from the training data- Usually 'Bovine' or 'Squid'
drop_ref = False

# Methos of encoding opsin sequences - Options: One-Hot-Encoding ('hot') or Amino-Acid Property Encoding ('aa_prop')
encoding_method='aa_prop'
use_best_props=True
# Specify which properties you want to keep for the amino-acid property encoding:
# We keep FIVE by deafult - 'H1, H3, P1, NCI, MASS' 
# But ELEVEN total are avaliable -'H1, H2, H3, P1, P2, V, NCI, MASS, SASA, PKA, PKB and SCT' 
# If you want to keep ALL aa props, just set props_to_keep = 'all'
# Or specify the properties in list format props_to_keep = ['H1', 'H3', 'P1', 'NCI', 'MASS']
# Only need to worry about this if encoding_method == 'aa_prop'
if use_best_props == True:
    props_to_keep = get_best_aa_prop_combos(dataset)
else:
    props_to_keep = ['H1', 'H3', 'NCI']

if use_gs_params == False:
    dataset='ignore'

In [ ]:
# making a unique directory for saving the reports of the analysis
print('direcory preparation')
dt_label = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
seqFile = seqFileName.split('/')[2]
#print(seqFile)
seqFile = seqFile.split('.')[0]+'.'+seqFile.split('.')[1]
#print(seqFile)
if encoding_method == 'hot':
    report_dir = str(seqFile +'_' + mt + '_' + dt_label)
elif encoding_method == 'aa_prop':
    props_used = ''
    for props in props_to_keep:
        props_used += props + '_'
    report_dir = str(props_used + seqFile +'_' + mt + '_' + dt_label)
os.makedirs(report_dir)

In [ ]:
%%time
print('reading meta-data')
# importing metadata
meta_data = read_data(metaDataFileName, seq_type = None, is_main=False)
# importing sequences data
print('reading fasta file')

tr = read_data(seqFileName, seq_type = seq_type, is_main=True, gap_threshold=gap_threshold)

shutil.copy2(f'{seqFileName}',report_dir)
write_fasta(dat = tr, fasta_file = f'{seqFile}_gap_dropped.fasta' , report_dir = report_dir)

In [ ]:
try:
    reference_seq = tr.loc['Bovine'].copy()
    ref_seq_name = 'bovine'
    if drop_ref == True:
        meta_data = meta_data.drop('Bovine')
    #print(bovine)
except:
    reference_seq = tr.loc['Squid'].copy()
    ref_seq_name = 'squid'
    #print(squid)
reference_seq.to_csv(path_or_buf= f'{report_dir}/ref_sequence.csv',index = True,mode="w")

In [ ]:
tr = tr.merge(meta_data.loc[:, mt],  left_index=True, right_index=True)
tr.shape

In [ ]:
y = tr.loc[:, mt].values
if use_ev == True:
    y = 1239.8 / np.array(y)
tr.drop(mt, axis=1, inplace=True)
print('Shape of data is: ', tr.shape)

**Attention**: metadata and sequences data should have the names as their row names and for each sequence their must be a value in the meta data file.

In [ ]:
print('metadata looks like this:')
meta_data.head()

In [ ]:
print('sequence data looks like this:')
tr.head()

### Preprocessing
In this step, we do all these steps:
1. dropping columns with a number of missing values above a certain threshold  
2. dropping zero entropy columns  
3. imputing missing values with the mode of that column  
4. replacing cases with a frequency below a threshold (default 1.5%) with the mode of that column
5. dropping zero entropy columns
6. use statistical tests (each position against the phenotype) and drop columns with p-values below a threshold (default 0.25)
7. one-hot encode the remaining columns
8. calculate the pair-wise distance matrix for all of the columns
9. use the distance matrix for DBSCAN and cluster the correlated positions together
10. keep only one column (closes to center of each cluster) for each group and drop the rest from the training data set

In [ ]:
if encoding_method == 'hot': 
    prep_pipeline = make_pipeline(
        steps=[
            ('mc', MisCare(missing_threshold=0.05)),
            ('cc', ConstantCare()),
            ('ur', URareCare(threshold=0.01)),
            ('cc2', ConstantCare()),
            ('one_hot', CustomOneHotEncoder()),
            ('feature_selection', FeatureSelection(model_type=ana_type, alpha=0.10, keep=False)),
            ('collinear_care', CollinearCare(dist_method='correlation', threshold=0.01, keep=False))
            
        ])
elif encoding_method == 'aa_prop':
    prep_pipeline = make_pipeline(
    steps=[
        ('mc', MisCare(missing_threshold=0.05)),
        ('cc', ConstantCare()),
        ('aa_prop', AminoAcidPropertyEncoder(props_to_keep = props_to_keep)),
        ('feature_selection', FeatureSelection(model_type=ana_type, alpha=0.10, keep=False)),
        ('collinear_care', CollinearCare(dist_method='correlation', threshold=0.01, keep=False))
    ])
else:
    raise ValueError('Encoding method not recognized')

In [ ]:
%%time
report, top = model_compare_cv(X=tr, y=y, preprocess_pipe=prep_pipeline,
                               models_dict=get_models(ana_type=ana_type, encoding=encoding_method, dataset=dataset),
                               scoring=get_scores(ana_type=ana_type),
                               report_dir=report_dir,
                               cv=10, ana_type=ana_type, cache_dir=report_dir)

MAE = Mean Absolute Error

MSE = Mean Squared Error

RMSE = Rooted Mean Square Error

MAPE = Mean Absolute % Error - the average magnitude of error produced by a model, or how far off predictions are on average. A MAPE value of 20% means that the average absolute percentage difference between the predictions and the actuals is 20%

In [ ]:
report

In [ ]:
if encoding_method == 'hot':
    prep_pipeline = make_pipeline(
        steps=[
            ('mc', MisCare(missing_threshold=0.05)),
            ('cc', ConstantCare()),
            ('ur', URareCare(threshold=0.025)),
            ('cc2', ConstantCare()),
            ('one_hot', CustomOneHotEncoder()),
            ('feature_selection', FeatureSelection(model_type=ana_type, alpha=0.10, keep=True)),
            ('collinear_care', CollinearCare(dist_method='correlation', threshold=0.01, keep=True))
        ])
elif encoding_method == 'aa_prop':
    prep_pipeline = make_pipeline(
        steps=[
            ('mc', MisCare(missing_threshold=0.05)),
            ('cc', ConstantCare()),
            ('aa_prop', AminoAcidPropertyEncoder(props_to_keep = props_to_keep)),
            ('feature_selection', FeatureSelection(model_type=ana_type, alpha=0.10, keep=True)),
            ('collinear_care', CollinearCare(dist_method='correlation', threshold=0.01, keep=True))
        ])
else:
    raise ValueError('Encoding method not recognized')

In [ ]:
modified_top = []
mtml = []
for model in top:
    modified_top.append(make_pipeline(steps=[('prep', prep_pipeline), model.steps[-1]]))
    my_top_models = str(model[1:])
    my_top_models = my_top_models.split("'")[3]
    mtml.append(my_top_models)
    #print(my_top_models)

In [ ]:
modified_top[0]

In [ ]:
%%time
top = finalize_top(X=tr, y=y, top_models=modified_top, grid_param=get_empty_params(),report_dir=report_dir, cv=10)


In [ ]:
%%time
sr = summarize_results(top_models=top, report_dir=report_dir)

In [ ]:
sr.head()

In [ ]:
scatter_plot = plot_scatter(summary_result=sr, report_dir=report_dir)

In [ ]:
%%time
mean_imp = mean_importance(top, report_dir=report_dir)

In [ ]:
if encoding_method == 'hot':
    dp_plot(importance=mean_imp,imp_col='mean', model_name='mean', report_dir=report_dir)
elif encoding_method == 'aa_prop':
    dp_aa_prop_plot(importance=mean_imp,imp_col='mean', model_name='mean', report_dir=report_dir, props_to_keep = props_to_keep)

In [ ]:
if use_ev == False:
    meta_var='λmax (nm)'
else:
    meta_var='λmax (eV)'
    
if encoding_method == 'hot':
    tr_copy = tr.copy()
    encoded_seqs = prep_pipeline[:4].fit_transform(tr)
    for model in top:
        model_name = model.steps[-1][0]
        dp_plot(importance=importance_from_pipe(model),
                imp_col='standard_value',
                model_name = model_name, report_dir=report_dir)
        
        plot_imp_model(importance=importance_from_pipe(model), 
                X_train=encoded_seqs, y_train=y, model_name=model_name,
                    meta_var=meta_var, model_type=ana_type, report_dir=report_dir)

    pl = plot_imp_all(final_models=top,
                    X_train=tr, y_train=y,
                    model_type = ana_type,
                    report_dir=report_dir, max_plots=350,
                    figsize=(2.5, 3), meta_var=meta_var)
elif encoding_method == 'aa_prop':
    for model in top:
        encoded_seqs = model.named_steps['prep']['aa_prop'].aa_encoded_seqs_
        model_name = model.steps[-1][0]
        
        dp_aa_prop_plot(importance=aaprop_importance_from_pipe(model),
                imp_col='standard_value',
                model_name = model_name, report_dir=report_dir, props_to_keep = props_to_keep)
        
        plot_imp_model(importance=aaprop_importance_from_pipe(model), 
                X_train=encoded_seqs, y_train=y, model_name=model_name,
                    meta_var='λmax', model_type=ana_type, report_dir=report_dir)
        
    pl = plot_imp_all(final_models=top,
                X_train=encoded_seqs, y_train=y,
                model_type = ana_type,
                report_dir=report_dir, max_plots=100,
                figsize=(2.5, 3), meta_var=meta_var)
    
        


# <font color=#c994c7>Step 4: Translate Candidate Spectral Tuning Sites (STS)</font> 
### This section is used to translate candidate STS to the bovine or squid equivalent.
 - The bovine and squid sequence dataframes that were saved earlier and are called again here


In [ ]:
from vpod_scripts.translate_candidate_sts import translate_candidate_sts, translate_candidate_sts_aa_props

if encoding_method == 'hot':
    trans_imp_report = translate_candidate_sts(report_dir, ref_seq_name, reference_seq)
elif encoding_method == 'aa_prop':
    trans_imp_report = translate_candidate_sts_aa_props(report_dir, reference_seq, ref_seq_name)

trans_imp_report.head()

# <font color=#c994c7>STEP 5: Query the Model to Predict NEW Sequences</font> 
### Takes new sequences, inserts them into existing alignment to properly format for model query, then returns prediction of the λmax value for each sequence...

In [ ]:
import os
import subprocess
from deepBreaks.utils import load_obj
from deepBreaks.preprocessing import read_data
import joblib
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from vpod_scripts.prediction_functions import process_sequences_from_file

This is a version of the prediction method which can be used DIRECTLY after model training... 

In [ ]:
#path to the mafft.bat file - change to your own directory!
mafft_exe = 'C:/Users/safra/mafft-win/mafft.bat'
#path to sequences we want to add to an existing alignment in FASTA format
input_file = './subtests/supp_test_data/msp_erg_raw.txt'
#name for desired output file
output_file = f'{report_dir}/ex_opsin_predictions.tsv'
#path to target/selected model
selected_model = report_dir + '/' + mtml[0] + '.pkl'
#function for querying model - this will take care of running predictions and creating an output file for you.
predictions_df = process_sequences_from_file(mafft_exe,input_file,output_file,selected_model,seqFileName, gap_threshold=gap_threshold)

In [ ]:
#load the meta-data file with our 'ground-truths'
msp_meta_file = './subtests/supp_test_data/msp_erg_meta.tsv'
msp_meta = read_data(msp_meta_file, seq_type = None, is_main=False)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score


# Create scatter plot to visulize relationship between our predicted and 'true' lmax values...
plt.scatter(msp_meta['Lambda_Max'], predictions_df['Predictions'])

# Add labels and title
plt.xlabel('True Values')
plt.ylabel('Predicted Values')
plt.title('Model Predictions vs True Values')

# Display plot
plt.show()

# Calculate R-squared
r2 = r2_score(msp_meta['Lambda_Max'], predictions_df['Predictions'])

# Print R-squared
print(f"R-squared: {r2:.3f}")